In [134]:
from collections import defaultdict
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

wrong, real = [], []
file = []

with open('queries_all.txt', 'rt') as f:    
    
    file = f.read().split('\n')
    for line in file:
        if '\t' in line:
            pair = line.split('\t')
            wrong.append(pair[0])
            real.append(pair[1])            
            
clear_dict = set()
word_freq = defaultdict(float)
cnt = 0

for line in real:
    
    words = set(line.lower().split())
    clear_dict.update(words)
                
    for word in words:
        word_freq[word] += 1
        cnt += 1
        
for word in word_freq.keys():
    word_freq[word] /= cnt

In [147]:
def get_score(correct_func, amount=100, return_flag=False):    
    """
    Функция, которая выдает 
    качество кор-ующей функции
    на wrong_real выборке.
    """
    global wrong, real    
    wrong_cnt = 0  
    real_cnt = 0
    if return_flag:
        wrong_positive = []
        wrong_negative = []
    
    np.random.seed(1337)
    wrong_indexes = np.random.randint(0, len(wrong), amount//2)
    real_indexes  = np.random.randint(0, len(real), amount//2)
    
    for i in range(amount//2):    
        
        idx = wrong_indexes[i]
        corrected = correct_func(wrong[idx])
        if corrected == real[idx]:
            wrong_cnt += 1
        else:            
            if return_flag:
                wrong_positive.append((wrong[idx], corrected, real[idx]))
            
        
        idx = real_indexes[i]
        corrected = correct_func(real[idx])
        if corrected == real[idx]:
            real_cnt += 1
        else:
            if return_flag:
                wrong_negative.append((real[idx], corrected))
            
    p = wrong_cnt / (amount//2) + 1e-9  # True positive rate
    r = real_cnt  / (amount//2) + 1e-9  # True negative rate
    
    if return_flag:
        wrong_negative.append(p)
        wrong_positive.append(r)        
        return 2 / (1/p + 1/r), wrong_negative, wrong_positive
    else:
        return 2 / (1/p + 1/r)

In [148]:
layout = {'1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '0': '0',
          '-': '-', '=': '=', 'q': 'й', 'w': 'ц', 'e': 'у', 'r': 'к', 't': 'е', 'y': 'н', 'u': 'г', 'i': 'ш',
          'o': 'щ', 'p': 'з', '[': 'х', ']': 'ъ', '\\':'\\','a': 'ф', 's': 'ы', 'd': 'в', 'f': 'а', 'g': 'п',
          'h': 'р', 'j': 'о', 'k': 'л', 'l': 'д', ';': 'ж', '\'':'э', 'z': 'я', 'x': 'ч', 'c': 'с', 'v': 'м',
          'b': 'и', 'n': 'т', 'm': 'ь', ',': 'б', '.': 'ю', '/': '.', '!': '!', '@': '"', '#': '№', '$': ';',
          '%': '%', '^': ':', '&': '?', '*': '*', '(': '(', ')': ')', '_': '_', '+': '+', 'Q': 'Й', 'W': 'Ц',
          'E': 'У', 'R': 'К', 'T': 'Е', 'Y': 'Н', 'U': 'Г', 'I': 'Ш', 'O': 'Щ', 'P': 'З', '{': 'Х', '}': 'Ъ',
          '|': '/', 'A': 'Ф', 'S': 'Ы', 'D': 'В', 'F': 'А', 'G': 'П', 'H': 'Р', 'J': 'О', 'K': 'Л', 'L': 'Д',
          ':': 'Ж', '"': 'Э', 'Z': 'Я', 'X': 'Ч', 'C': 'С', 'V': 'М', 'B': 'И', 'N': 'Т', 'M': 'Ь', '<': 'Б',
          '>': 'Ю', '?': ',', ' ': ' ',
          
          '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '0': '0',
          '-': '-', '=': '=', 'й': 'q', 'ц': 'w', 'у': 'e', 'к': 'r', 'е': 't', 'н': 'y', 'г': 'u', 'ш': 'i',
          'щ': 'o', 'з': 'p', 'х': '[', 'ъ': ']', '\\':'\\','ф': 'a', 'ы': 's', 'в': 'd', 'а': 'f', 'п': 'g',
          'р': 'h', 'о': 'j', 'л': 'k', 'д': 'l', 'ж': ';', 'э': "'", 'я': 'z', 'ч': 'x', 'с': 'c', 'м': 'v',
          'и': 'b', 'т': 'n', 'ь': 'm', 'б': ',', 'ю': '.', '.': '/', '!': '!', '"': '@', '№': '#', ';': '$',
          '%': '%', ':': '^', '?': '&', '*': '*', '(': '(', ')': ')', '_': '_', '+': '+', 'Й': 'Q', 'Ц': 'W',
          'У': 'E', 'К': 'R', 'Е': 'T', 'Н': 'Y', 'Г': 'U', 'Ш': 'I', 'Щ': 'O', 'З': 'P', 'Х': '{', 'Ъ': '}',
          '/': '|', 'Ф': 'A', 'Ы': 'S', 'В': 'D', 'А': 'F', 'П': 'G', 'Р': 'H', 'О': 'J', 'Л': 'K', 'Д': 'L',
          'Ж': ':', 'Э': '"', 'Я': 'Z', 'Ч': 'X', 'С': 'C', 'М': 'V', 'И': 'B', 'Т': 'N', 'Ь': 'M', 'Б': '<',
          'Ю': '>', ',': '?',
         
          '`': 'ё', '~': 'Ё', 'ё': '`', 'Ё': '~'}



def change_layout(word):
    
    return ''.join([layout[i] if i in layout else '*' for i in word])

In [9]:
def dl_dist(unknown_word, true_word):

    def damerau_levenshtein_distance(s1, s2):
        """
        damerau_levenshtein_distance
        """
        d = {}
        lenstr1 = len(s1)
        lenstr2 = len(s2)
        for i in range(-1,lenstr1+1):
            d[(i,-1)] = i+1
        for j in range(-1,lenstr2+1):
            d[(-1,j)] = j+1

        for i in range(lenstr1):
            for j in range(lenstr2):
                if s1[i] == s2[j]:
                    cost = 0
                else:
                    cost = 1
                d[(i,j)] = min(
                               d[(i-1,j)] + 1, # deletion
                               d[(i,j-1)] + 1, # insertion
                               d[(i-1,j-1)] + cost, # substitution
                              )
                if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                    d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition

        return d[lenstr1-1,lenstr2-1]
    
    return min(damerau_levenshtein_distance(unknown_word, true_word),
               damerau_levenshtein_distance(change_layout(unknown_word), true_word) + 1)

In [35]:
# get_score(correct_line)

In [11]:
correct_it('пивет')

'пишет привет живет пивот'

In [206]:
class Trie:

    def __init__(self, rm_p=0.8, rp_p=0.7, in_p=0.6):
        self.children = {}
        self.word = ''
        self.isWord = False
        self.remove_p  = rm_p
        self.replace_p = rp_p
        self.insert_p  = in_p

        
    def insert(self,  word, level=0):
        """
        Recursion insert of new word in Trie
        """
        if not word:
            return

        if word[level] not in self.children:
            self.children[word[level]] = Trie()        
        
        if word[level+1:]:
            self.children[word[level]].insert(word, level+1)
            
        else:
            self.children[word[level]].word = word
            self.children[word[level]].isWord = True

            
    def find(self, word, level=0, p=0.9999):
        """
        Returns list of closest words in Trie
        """        
        if p < 0.45:#min(max(max(self.remove_p, self.replace_p, self.insert_p), 0.4), 0.7):
            return []
        
        if not word[level:]:

            if self.isWord:
                return [(self.word, word_freq[self.word])]
                
            else:
                return []
        
        res = []
        
        for count, letter in enumerate(self.children.keys()):
            
            if word[level] == letter:
                res += self.children[letter].find(word, level+1, p)
            
            else:
                res += self.children[letter].find(word, level+2, p*self.remove_p)
                res += self.children[letter].find(word, level+1, p*self.replace_p)
                res += self.children[letter].find(word, level,   p*self.insert_p)
                
        return res

In [190]:
result_df.sort_values('f1', ascending=False).head()

,f1,insert,remove,replace
76,0.234667,0.618696,0.776264,0.733768
23,0.234667,0.678450,0.891627,0.810490
54,0.234667,0.831097,0.730643,0.813064
28,0.234667,0.609372,0.698628,0.720331
30,0.234667,0.735492,0.856048,0.740026


In [207]:
tree = Trie()
for word in clear_dict:
    tree.insert(word)

In [204]:
def correct_it(word, logging=False):
    
    cur_max = -1
    ans = ''
    search_list = tree.find(word)
    search_list.extend(tree.find(change_layout(word)))
    
    for pair in search_list:
        if pair[1] > cur_max:
            cur_max = pair[1]
            ans = pair[0]
            
    if logging:
        for elem in search_list: print(elem)
    
    return ans#, cur_max


def correct_line(line):
    
    return ' '.join(correct_it(word) for word in line.split())        

In [205]:
score, wn_list, wp_list = get_score(correct_line, 100, True)
score

0.18947368521329638

In [196]:
score, wn_list, wp_list = get_score(correct_line, 100, True)
score

0.18947368521329638

In [186]:
result_df = pd.DataFrame([])
tree = Trie()
for word in clear_dict:
    tree.insert(word)
    
scope = np.random.rand(100, 3) * 0.4 + 0.6
for i in tqdm(range(100)):
    rm_p = scope[i,0]
    rp_p = scope[i,1]
    in_p = scope[i,2]
    
    tree.remove_p  = rm_p
    tree.replace_p = rp_p
    tree.insert_p  = in_p

    score = get_score(correct_line, 50)

    result_df = result_df.append({'f1': score,
                                  'remove' : rm_p,
                                  'replace': rp_p,
                                  'insert' : in_p}, ignore_index=True)

KeyboardInterrupt: 

In [187]:
result_df.sort_values('f1')

,f1,insert,remove,replace
38,0.064000,0.845948,0.901222,0.799966
41,0.064000,0.744644,0.984553,0.836104
43,0.064000,0.769917,0.901873,0.893806
44,0.064000,0.854812,0.904203,0.624271
50,0.064000,0.958054,0.933587,0.883188
...,...,...,...,...
53,0.234667,0.729026,0.650296,0.699823
54,0.234667,0.831097,0.730643,0.813064
55,0.234667,0.741495,0.669944,0.692269
47,0.234667,0.726787,0.809643,0.633648


In [206]:
# %%timeit
# correct_it('привет', logging=False)

KeyboardInterrupt: 

In [154]:
tree = Trie()

In [155]:
for word in clear_dict:
    tree.insert(word)

In [158]:
cur_max = -1
search_list = tree.search_relevant('привет')
for a in search_list:
    if a[1] > cur_max:
        cur_max = a[1]
        ans = a[0]

print(ans, "%.7f" % cur_max, end='\n\n')
for elem in search_list:
    print(elem)

привет, 0.0000045

('привет!', 2.273424800791152e-06)
('привет)', 2.273424800791152e-06)
('привет,', 4.546849601582304e-06)


In [262]:
'привет' in clear_dict

True

In [265]:
%%time
correct_it('пивет')

CPU times: user 14.7 s, sys: 1.59 ms, total: 14.7 s
Wall time: 14.7 s


'пивот живет пишет привет'